In [3]:
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import torch.optim as optim

import json 


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
binary_path: c:\Python311\Lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll
CUDA SETUP: Loading binary c:\Python311\Lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll...


In [4]:

# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set padding token
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)

# IMPORTANT: Update the padding token ID in the model configuration
model.config.pad_token_id = model.config.eos_token_id


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
import codecs

# This file was generate with GPT-4. I asked it to randomly generate answers to a set of questions, and then pick a preferred answer for the annotation. 
# This would have to be replaced by the desired dataset and the 'answer1' and 'answer2' annotations may be changed. If changed, I need to update the following cell where the labels are extracted.
data_path = './dataset_reward_model.json'

with codecs.open(data_path, 'r', encoding='utf-8') as f:
      dataset = json.load(f)

In [6]:
# Prepare data
prompts = [item['data']['prompt'] for item in dataset]
answer1s = [item['data']['answer1'] for item in dataset]
answer2s = [item['data']['answer2'] for item in dataset]
labels_list = [0 if item['data']['annotation'] == 'answer1' else 1 for item in dataset]

In [7]:
# Tokenization
max_length = 100  # Choose a max_length that fits your data
encodings = tokenizer(prompts, answer1s, answer2s, truncation=True, padding='max_length', max_length=max_length)
input_ids = torch.tensor(encodings['input_ids'])
attention_mask = torch.tensor(encodings['attention_mask'])
labels = torch.tensor(labels_list)
labels = labels.clone().detach()

In [8]:
# DataLoader
dataset = TensorDataset(input_ids, attention_mask, labels)
loader = DataLoader(dataset, batch_size=2, shuffle=True)  # Use an appropriate batch_size

In [9]:
# Training setup
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
epochs = 3

# Training loop
for epoch in range(epochs):
    for i, batch in enumerate(loader):
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        
        print(f"Epoch {epoch+1}/{epochs}, Batch {i+1}/{len(loader)}, Loss: {loss.item()}")


Epoch 1/3, Batch 1/16, Loss: 0.3626187741756439
Epoch 1/3, Batch 2/16, Loss: 0.0004944835091009736
Epoch 1/3, Batch 3/16, Loss: 8.265109062194824
Epoch 1/3, Batch 4/16, Loss: 0.0030704401433467865
Epoch 1/3, Batch 5/16, Loss: 4.266773223876953
Epoch 1/3, Batch 6/16, Loss: 0.0013885501539334655
Epoch 1/3, Batch 7/16, Loss: 0.003802233375608921
Epoch 1/3, Batch 8/16, Loss: 0.25442084670066833
Epoch 1/3, Batch 9/16, Loss: 3.107051134109497
Epoch 1/3, Batch 10/16, Loss: 12.563957214355469
Epoch 1/3, Batch 11/16, Loss: 6.96302604675293
Epoch 1/3, Batch 12/16, Loss: 3.980490207672119
Epoch 1/3, Batch 13/16, Loss: 8.177164077758789
Epoch 1/3, Batch 14/16, Loss: 3.5492444038391113
Epoch 1/3, Batch 15/16, Loss: 5.57310676574707
Epoch 1/3, Batch 16/16, Loss: 3.62490177154541
Epoch 2/3, Batch 1/16, Loss: 0.354430228471756
Epoch 2/3, Batch 2/16, Loss: 2.85650897026062
Epoch 2/3, Batch 3/16, Loss: 2.8355190753936768
Epoch 2/3, Batch 4/16, Loss: 0.029429355636239052
Epoch 2/3, Batch 5/16, Loss: 2.93

In [10]:
# Save the model to a directory
save_directory = "model_rewards_model"
model.save_pretrained(save_directory)

# Optionally, save the tokenizer as well, especially if you've added special tokens or made other changes
tokenizer.save_pretrained(save_directory)


('model_rewards_model\\tokenizer_config.json',
 'model_rewards_model\\special_tokens_map.json',
 'model_rewards_model\\vocab.json',
 'model_rewards_model\\merges.txt',
 'model_rewards_model\\added_tokens.json')

In [13]:
import torch.nn.functional as F

def calc_reward(model, tokenizer, prompt, answer1, answer2):
    # Tokenize the input
    inputs = tokenizer(prompt, [answer1, answer2], return_tensors='pt', padding=True, truncation=True, max_length=100)

    # Get model output
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits

    # Calculate probabilities
    probs = F.softmax(logits, dim=-1)

    # Interpret the result
    return logits 

    if probs[0, 0] > probs[0, 1]:
        print(f"The model prefers '{answer1}' with a probability of {probs[0, 0]:.4f}")
    else:
        print(f"The model prefers '{answer2}' with a probability of {probs[0, 1]:.4f}")

In [15]:
# Test the function
prompt = "What are the latest developments in artificial intelligence?"
answer1 = "GANs are revolutionizing image creation, and NLP models like GPT-3 are transforming language tasks."
answer2 = "AI is making strides in healthcare for diagnosis, and reinforcement learning is advancing robotics."

logits = calc_reward(model, tokenizer, prompt, answer1, answer2)
print(logits)

tensor([[1.6558, 6.6592]])
